# Libraries & Path Setting 

In [1]:
import sys, os
import tensorflow as tf
import tensorflow_hub as hub
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

/data/ontology/anaconda3/envs/conda_env_murali/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sys.path.append("..")
os.environ["CUDA_VISIBLE_DEVICES"]='2';
tf_sess_config = tf.ConfigProto()
tf_sess_config.gpu_options.allow_growth = True

In [ ]:
!ls

# Analyze at word level

In [3]:
tf.reset_default_graph();
elmo = hub.Module("../Data/elmo2", trainable=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


In [ ]:
def ElmoEmbedding_v1(sent_list): 
    return elmo(sent_list,signature="default",as_dict=True)["elmo"]

def ElmoEmbedding_v2(tokens_input, tokens_length): 
    return elmo(inputs={"tokens": tokens_input,"sequence_len": tokens_length},signature="tokens",as_dict=True)["elmo"]     

In [ ]:
sents = ['My samsung galaxy next has charging problem', 
         'me samsung galxy nxt hs charging probs'];
with tf.Session(config=tf_sess_config, graph=tf.get_default_graph()) as sess:
    sess.run(tf.global_variables_initializer());
    embs_ = ElmoEmbedding_v1(sents);
    embs = sess.run([embs_])[0];  

In [ ]:
sents = ['Why device is showing full storage after software update is it consuming lots of space of device.', 
         'y dvice is shwing fll strg aftr sftr updt / @ consuming lts off spce of device.'];
with tf.Session(config=tf_sess_config, graph=tf.get_default_graph()) as sess:
    sess.run(tf.global_variables_initializer());
    embs_ = ElmoEmbedding_v1(sents);
    embs = sess.run([embs_])[0];  

In [ ]:
def similarity(X,Y):
    return cosine_similarity(X,Y)

sim_ = similarity(embs[0][:],embs[1][:])
sim = np.around(sim_, decimals=1)
print(sim)

In [ ]:
fig, ax = plt.subplots(1, 1)
c = ax.pcolor(sim)
ax.set_title('color_gradient');
fig.tight_layout()
plt.show()

# Analyze at Senetence Level

In [64]:
# List 
sents = ['','',''];

In [65]:
def cleanText_Spaces_(sents):
    new_sents = [];
    new_sents_lens = [];
    for sent in sents:
        tokens = [x for x in re.split('\W+', sent) if x]
        new_sents.append(tokens)
        new_sents_lens.append(len(tokens))
    return new_sents, new_sents_lens

In [66]:
def sents_for_elmo_input(new_sents, new_sents_lens):
    max_len = np.max(new_sents_lens)
    for i, token_list in enumerate(new_sents):
        new_sents[i]+=[""]*(max_len-new_sents_lens[i])
    return np.vstack(new_sents), np.stack(new_sents_lens)

In [67]:
new_sents, new_sents_lens = cleanText_Spaces_(sents)
print(new_sents, new_sents_lens)
print('\n')
new_new_sents, new_sents_lens = sents_for_elmo_input(new_sents, new_sents_lens)
print(new_new_sents.shape, new_sents_lens.shape)

[['What', 'is', 'Menu', 'Transparency'], ['Need', 'info', 'about', 'menu', 'transparency'], ['I', 'want', 'to', 'make', 'a', 'reservation', 'in', 'an', 'Italian', 'restaurant'], ['I', 'need', 'a', 'table', 'in', 'a', 'pizzeria'], ['I', 'donot', 'want', 'to', 'go', 'to', 'movies'], ['I', 'hate', 'watching', 'films', 'at', 'theaters'], ['I', 'love', 'watching', 'movies'], ['i', 'wanna', 'enjoy', 'weekend', 'with', 'a', 'cinema'], ['How', 'to', 'connect', 'an', 'MHL', 'to', 'HDMI', 'cable', 'in', 'TV'], ['Possible', 'way', 'to', 'join', 'hdmi', 'to', 'mhl', 'cable'], ['Guide', 'me', 'with', 'the', 'steps', 'to', 'connect', 'mhl', 'and', 'hdmi', 'cable'], ['What', 'to', 'do', 'when', 'the', 'DVD', 'is', 'getting', 'No', 'Disc', 'error'], ['Disk', 'is', 'not', 'read', 'by', 'DVD', 'player'], ['I', 'am', 'facing', 'disc', 'detection', 'issue', 'in', 'dvd', 'player']] [4, 5, 10, 7, 7, 6, 4, 7, 10, 8, 11, 11, 7, 9]


(14, 11) (14,)


In [68]:
tf.reset_default_graph();

class ELMo_AvgPool(object):
    def __init__(self, elmo_module):
        self.elmo_module = elmo_module;
        self.inp_tokens = tf.placeholder(tf.string, shape=(None, None));
        self.inp_tokens_listlen = tf.placeholder(tf.int32, shape=(None,));
        self.embs__ = self.Elmo_SentEmbedding(self.inp_tokens, self.inp_tokens_listlen);
        self.masks = tf.cast(tf.tile(tf.expand_dims(tf.sequence_mask(self.inp_tokens_listlen),axis=-1),[1,1,tf.shape(self.embs__)[-1]]),tf.float32)
        self.embs_ = tf.multiply(self.embs__,self.masks)
        self.embs = tf.reduce_mean(self.embs_, axis=-2);
    def Elmo_SentEmbedding(self, tokens_input, tokens_lengths):
        return self.elmo_module(inputs={"tokens": tokens_input,"sequence_len": tokens_lengths},signature="tokens",as_dict=True)["elmo"]#shape [bs,max_len,1024]   

simple_graph = tf.Graph();
with simple_graph.as_default():
    elmo_module = hub.Module("../Data/elmo2", trainable=True)
    elmo_avg_pool = ELMo_AvgPool(elmo_module)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [69]:
with tf.Session(config=tf_sess_config, graph=simple_graph) as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    embs, embs_, masks = sess.run([elmo_avg_pool.embs, elmo_avg_pool.embs_, elmo_avg_pool.masks],
                           feed_dict={elmo_avg_pool.inp_tokens:new_new_sents,
                                      elmo_avg_pool.inp_tokens_listlen:new_sents_lens});

In [70]:
#a = np.dot(embs,embs.T)
a = cosine_similarity(embs,embs)
b = a.max(axis=0)
c = a/b
d = c-np.eye(c.shape[0])
e = np.exp(d)
f = e/np.sum(e,axis=1)
g = np.around(f, decimals=2)
h = np.argmax(g, axis=-1)
for i,sent in enumerate(sents):
    print(sent+' ==> '+sents[h[i]])

What is Menu Transparency?  ==> Need info about menu transparency.
Need info about menu transparency. ==> What is Menu Transparency? 
I want to make a reservation in an Italian restaurant ==> I need a table in a pizzeria
I need a table in a pizzeria ==> I want to make a reservation in an Italian restaurant
I donot want to go to movies ==> I hate watching films at theaters
I hate watching films at theaters ==> I love watching movies
I love watching movies ==> I hate watching films at theaters
i wanna enjoy weekend with a cinema ==> I want to make a reservation in an Italian restaurant
How to connect an MHL to HDMI cable in TV? ==> Possible way to join hdmi to mhl cable?
Possible way to join hdmi to mhl cable? ==> Guide me with the steps to connect mhl and hdmi cable?
Guide me with the steps to connect mhl and hdmi cable? ==> How to connect an MHL to HDMI cable in TV?
What to do when the DVD is getting No Disc error? ==> What is Menu Transparency? 
Disk is not read by DVD player. ==> How